# Simulation of filaments

This notebook tests the simulation of filaments by Monte-Carlo using the 'mixed model'


The normalized model is

In [7]:
import numpy as np
import scipy.optimize as spo
import scipy.integrate as spi
import matplotlib.pyplot as plt

In [34]:
# Helper functions

def machnumber(x):
    assert((x>=0) & (x<=1))
    f = lambda M: 2*np.arctan(M)-M-(0.5*np.pi-1)*x
    return spo.root_scalar(f,bracket=[0,1]).root

def tmax(x0,t0):
    # Find the time a particle starting at x(t0)=x0 reaches x=1
    
    # We have dx/dt = M(x), and the particles should disappear at x=1.
    # To find the time for a particle starting at X(t0)=x0 with M(x0)=M0, 
    # we rather integrate dt/dx=1/M(x) with the solution below

    M0 = machnumber(x0)
    return t0+np.log(0.5*(M0+1/M0))/(0.5*np.pi-1)

def sample_from_density():
    # We have that the LCFS density is n = 1/(1+M^2), so 

In [38]:
class SimulationParams:
    def __init__(self,dt=0.1):
        self.dt = dt

class Particle:
    def __init__(self,t0, x0, SimulationParams):
        self.params = SimulationParams

        # make time array up to max time
        self.T = np.arange(t0, tmax(x0,t0), self.params.dt)

        # Solve dx/dt = M(x)
        self.X = spi.solve_ivp(lambda t,x: machnumber(x[0]), 
                               (self.T[0],self.T[-1]),
                               np.array([x0,]), t_eval = self.T).y[0]

        # Solution to dR/dt = 1 with R(0)=0
        self.R = self.T-self.T[0]

    def gen_machnumber(self):
        self.M = np.zeros_like(self.T)
        for i in range(self.T.size):
            self.M[i] = machnumber(self.X[i])